In [5]:
import csv
import math
import random
import time
# -------------------------
# Helper Functions
# -------------------------
def read_csv(filename):
    """Read a CSV file into a list of lists of floats."""
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        return [[float(x) for x in row] for row in reader]

def euclidean_distance(a, b):
    return math.sqrt(sum((a[i] - b[i]) ** 2 for i in range(len(a))))

def cosine_distance(a, b):
    num = sum(a[i] * b[i] for i in range(len(a)))
    denom_a = math.sqrt(sum(a[i] ** 2 for i in range(len(a))))
    denom_b = math.sqrt(sum(b[i] ** 2 for i in range(len(b))))
    if denom_a == 0 or denom_b == 0:
        return 1.0
    return 1 - (num / (denom_a * denom_b))

def generalized_jaccard_distance(a, b):
    min_sum = sum(min(a[i], b[i]) for i in range(len(a)))
    max_sum = sum(max(a[i], b[i]) for i in range(len(a)))
    if max_sum == 0:
        return 1.0
    return 1 - (min_sum / max_sum)

def normalize_data(X):
    """Normalize dataset values to range [0, 1] using min-max scaling."""
    # Find global min and max
    min_val = min(min(row) for row in X)
    max_val = max(max(row) for row in X)
    if max_val == min_val:
        return X  # avoid division by zero

    # Scale each element
    normalized = []
    for row in X:
        normalized.append([(x - min_val) / (max_val - min_val) for x in row])
    return normalized



In [6]:

# -------------------------
# K-Means Implementation
# -------------------------
class KMeansScratch:
    def __init__(self, n_clusters=10, max_iter=100, distance='euclidean', criteria="centroid", random_state=42):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.stop_method = criteria
        random.seed(random_state)
        
        # Choose distance function
        if distance == 'euclidean':
            self.dist_func = euclidean_distance
        elif distance == 'cosine':
            self.dist_func = cosine_distance
        elif distance == 'jaccard':
            self.dist_func = generalized_jaccard_distance
        else:
            raise ValueError("distance must be 'euclidean', 'cosine', or 'jaccard'")
    
    def fit(self, X):
        start_time = time.time()
        n_features = len(X[0])
        current_compute_sse = float('inf')
        pre_compute_sse = float('inf')
        
        # Randomly initialize centroids
        self.centroids = random.sample(X, self.n_clusters)
        
        for iteration in range(self.max_iter):
            # Assign points to clusters
            clusters = [[] for _ in range(self.n_clusters)]
            for x in X:
                distances = [self.dist_func(x, c) for c in self.centroids]
                cluster_index = distances.index(min(distances))
                clusters[cluster_index].append(x)
            
            # Compute new centroids
            new_centroids = []
            for cluster in clusters:
                if not cluster:
                    # Empty cluster → keep previous centroid
                    new_centroids.append(random.choice(X))
                else:
                    # Mean of cluster points (element-wise)
                    new_centroid = []
                    for j in range(n_features):
                        col_sum = sum(point[j] for point in cluster)
                        new_centroid.append(col_sum / len(cluster))
                    new_centroids.append(new_centroid)
            
            self.labels_ = [self._closest_centroid(x) for x in X]
            current_compute_sse = self._compute_sse(X)
            # Check for convergence
            if self._converged(self.centroids, new_centroids,
                           current_sse=current_compute_sse, prev_sse=pre_compute_sse,
                           iteration=iteration, max_iter=self.max_iter):
                
                break

            pre_compute_sse = current_compute_sse
            
            self.centroids = new_centroids
        
        # Store final cluster assignments and compute SSE
        
        self.inertia_ = current_compute_sse

        self.iterations_ = iteration + 1
        self.time_ = time.time() - start_time
    
    def _closest_centroid(self, x):
        distances = [self.dist_func(x, c) for c in self.centroids]
        return distances.index(min(distances))
    
    def _compute_sse(self, X):
        sse = 0.0
        for i, x in enumerate(X):
            centroid = self.centroids[self.labels_[i]]
            dist = self.dist_func(x, centroid)
            sse += dist ** 2
        return sse
    
    def _converged(self, old_centroids, new_centroids, current_sse=None, prev_sse=None,
               iteration=None, max_iter=None, tol=1e-6):
        """
        Determine if K-Means should stop:
        1. Centroid movement below tolerance
        2. SSE stops improving (increasing)
        3. Reached maximum iteration count
        """

        # --- Centroid-based stopping ---
        total_change = 0.0
        for i in range(len(old_centroids)):
            total_change += euclidean_distance(old_centroids[i], new_centroids[i])
        if self.stop_method == 'centroid' and total_change < tol:
            print(f"Converged at iteration {iteration + 1} (centroids stabilized).")
            return True

        # --- SSE-based stopping ---
        if self.stop_method == 'sse' and current_sse is not None and prev_sse is not None:
            if current_sse >= prev_sse:
                print(f"Stopping early at iteration {iteration + 1} (SSE stopped improving).")
                return True

        # --- Maximum iteration stopping ---
        if (self.stop_method == "max_iteration") & (max_iter is not None and iteration + 1 >= max_iter):
            print(f"Reached maximum iterations ({max_iter}). Stopping.")
            return True

        return False



In [7]:
from collections import Counter

def compute_cluster_accuracy(labels, true_labels, k):
    """
    Compute clustering accuracy using majority voting within each cluster.
    labels: predicted cluster IDs (from KMeans)
    true_labels: ground-truth class labels
    k: number of clusters
    """
    cluster_to_label = {}
    correct = 0

    for cluster_id in range(k):
        cluster_indices = [i for i, lbl in enumerate(labels) if lbl == cluster_id]
        if not cluster_indices:
            continue
        cluster_true_labels = [true_labels[i] for i in cluster_indices]
        most_common = Counter(cluster_true_labels).most_common(1)[0][0]
        cluster_to_label[cluster_id] = most_common

    for i, cluster_id in enumerate(labels):
        predicted_label = cluster_to_label.get(cluster_id, -1)
        if predicted_label == true_labels[i]:
            correct += 1

    accuracy = correct / len(labels)
    return accuracy

In [8]:
# -------------------------
# Main Execution
# -------------------------
if __name__ == "__main__":
    # Load data
    data = read_csv('data.csv')
    labels = [int(x[0]) for x in read_csv('label.csv')]

    use_subdata = 1000
    
    print("Data samples:", use_subdata)
    print("Features per sample:", len(data[0]))

    # Normalize data
    normalized_data = normalize_data(data[:use_subdata])
    print("Data normalized to [0, 1]")
    
    subset_labels = labels[:use_subdata]
    

    for metric in ['euclidean', 'cosine', 'jaccard']:
        print("-"*40)
        print(f"Running K-Means with {metric} ")

        # there are three criteria: "centroid", "sse", "max_iteration"
        kmeans = KMeansScratch(n_clusters=10, max_iter=35, distance=metric, criteria="centroid")
        kmeans.fit(normalized_data)

        print(f"{metric.capitalize()} SSE: {kmeans.inertia_:.4f}")

        acc = compute_cluster_accuracy(kmeans.labels_, subset_labels, 10)
        print(f"{metric.capitalize()} K-Means Accuracy: {acc * 100:.2f}%")

        print(f"Iterations: {kmeans.iterations_}")
        print(f"Runtime: {kmeans.time_:.2f} seconds")


        


Data samples: 1000
Features per sample: 784
Data normalized to [0, 1]
----------------------------------------
Running K-Means with euclidean 
Converged at iteration 18 (centroids stabilized).
Euclidean SSE: 37082.3819
Euclidean K-Means Accuracy: 59.60%
Iterations: 18
Runtime: 18.64 seconds
----------------------------------------
Running K-Means with cosine 
Converged at iteration 23 (centroids stabilized).
Cosine SSE: 74.3334
Cosine K-Means Accuracy: 59.50%
Iterations: 23
Runtime: 47.43 seconds
----------------------------------------
Running K-Means with jaccard 
Converged at iteration 23 (centroids stabilized).
Jaccard SSE: 382.3725
Jaccard K-Means Accuracy: 56.30%
Iterations: 23
Runtime: 54.07 seconds
